In [1]:

import utils
import os

import torch
import torch_directml
import model.net as net
from gensim.models import KeyedVectors

seed = 230

# select the GPU device if available
# if torch.cuda.is_available():
#     device = torch.device("cuda")
#     torch.cuda.manual_seed(seed)
# elif torch_directml.is_available():
#     device = torch_directml.device(torch_directml.default_device())
# else:
#     device = torch.device("cpu")

device = torch.device("cpu")
torch.manual_seed(seed)

In [2]:
model_dir = os.path.join('experiments', 'base_model')

In [3]:
embeddings_kv = KeyedVectors.load_word2vec_format(
    "./data/embeddings/glove.6B.100d.txt", binary=False, no_header=True
)
embeddings = torch.tensor(embeddings_kv.vectors)

In [4]:
params = utils.Params(os.path.join(model_dir, 'params.json'))
checkpoint = torch.load(os.path.join(model_dir, 'best.pth.tar'))

model = net.Net(
    device=device,
    embeddings=embeddings,
    num_heads=params.num_heads,
    num_layers=params.num_layers,
    num_classes=params.num_classes,
    input_window_size=params.max_input_length,
).to(device)

In [5]:
# Remember that you must call model.eval() to set dropout and batch normalization
# layers to evaluation mode before running inference. Failing to do this will yield inconsistent inference results.
model.eval()

Net(
  (embedding): Embedding(400000, 100)
  (encoders): ModuleList(
    (0-1): 2 x TransformerEncoder(
      (heads): ModuleList(
        (0-9): 10 x AttentionHead(
          (w_q): Linear(in_features=100, out_features=10, bias=True)
          (w_k): Linear(in_features=100, out_features=10, bias=True)
          (w_v): Linear(in_features=100, out_features=10, bias=True)
          (softmax): Softmax(dim=-1)
          (fc): Linear(in_features=10, out_features=100, bias=True)
        )
      )
      (linear): Linear(in_features=1000, out_features=100, bias=True)
      (ln1): LayerNorm()
      (drop1): Dropout(p=0.1, inplace=False)
      (ff1): Linear(in_features=100, out_features=400, bias=True)
      (ff2): Linear(in_features=400, out_features=100, bias=True)
      (ln2): LayerNorm()
      (drop2): Dropout(p=0.1, inplace=False)
    )
  )
  (fc): Linear(in_features=100, out_features=2, bias=True)
)

In [6]:
def tokenize(input: list[str]) -> torch.LongTensor:
    xs = []
    for s in input:
        ws = [
            embeddings_kv.key_to_index[w]
            for w in s.split(" ")
            if w in embeddings_kv
        ]
        # pad with dots (must pad for input to be accepted; since inputs must be of same size)
        ws += [embeddings_kv.key_to_index["."]] * (params.max_input_length - len(ws))
        xs.append(ws[:params.max_input_length])
    return torch.LongTensor(xs)

In [7]:
input = [
    # Positive
    'This movie was absolutely amazing! The story was so gripping and the actors gave stellar performances. I highly recommend it.',
    'My friend cooked the most delicious dinner last night. The lasagna was made with fresh ingredients and tons of love. I left feeling so satisfied and happy.',
    'My new puppy brings me so much joy every day. His curious nature and endless energy are truly uplifting. He makes me smile no matter what.',

    # Negative
    'That was the worst customer service I\'ve ever experienced. The staff was rude and unhelpful. I\'ll never shop there again.',
    'Traffic was backed up for miles today. I wasted over an hour just sitting in my car, annoyed and frustrated.',
    'This sunset is so dull and boring. The sky is just gray and lifeless with no color whatsoever. What a letdown.',
]

In [8]:
x = tokenize(input)

In [9]:
with torch.no_grad():
    output = model(x)
predictions = output.argmax(1)

In [10]:
output

tensor([[0.0061, 0.0035],
        [0.0060, 0.0035],
        [0.0060, 0.0034],
        [0.0061, 0.0036],
        [0.0060, 0.0035],
        [0.0061, 0.0035]])

In [11]:
predictions

tensor([0, 0, 0, 0, 0, 0])